In [36]:
import numpy as np
import pandas as pd
import time
import os
import math
import threading
from concurrent import futures
import tushare as ts

In [37]:
print(ts.__version__)
pro = ts.pro_api()

1.2.60


In [38]:
stock_list = pro.stock_basic(exchange='', list_status='L')

In [39]:
stock_list.shape

(3893, 7)

In [40]:
total_stock_ts_code = list(stock_list['ts_code'])
total_stock_ts_code[0]

'000001.SZ'

In [41]:
df = pro.daily(trade_date = '20200709')

In [42]:
df.shape

(3856, 11)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3856 entries, 0 to 3855
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ts_code     3856 non-null   object 
 1   trade_date  3856 non-null   object 
 2   open        3856 non-null   float64
 3   high        3856 non-null   float64
 4   low         3856 non-null   float64
 5   close       3856 non-null   float64
 6   pre_close   3856 non-null   float64
 7   change      3856 non-null   float64
 8   pct_chg     3856 non-null   float64
 9   vol         3856 non-null   float64
 10  amount      3856 non-null   float64
dtypes: float64(9), object(2)
memory usage: 331.5+ KB


In [44]:
total_hist = pro.daily(ts_code='000001.SZ', start_date='20040101', end_date='20200711')

In [45]:
total_hist.sort_values(by=['trade_date'], inplace=True)
total_hist

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
3863,000001.SZ,20040102,8.51,8.65,8.41,8.64,8.51,0.13,1.5300,58047.80,4.963405e+04
3862,000001.SZ,20040105,8.64,8.98,8.60,8.92,8.64,0.28,3.2400,120577.77,1.065291e+05
3861,000001.SZ,20040106,9.00,9.55,9.00,9.33,8.92,0.41,4.6000,375917.45,3.515232e+05
3860,000001.SZ,20040107,9.33,9.53,9.25,9.39,9.33,0.06,0.6400,195362.44,1.834584e+05
3859,000001.SZ,20040108,9.40,9.47,9.23,9.41,9.39,0.02,0.2100,127015.27,1.184990e+05
...,...,...,...,...,...,...,...,...,...,...,...
4,000001.SZ,20200706,14.60,15.68,14.59,15.68,14.25,1.43,10.0351,4711460.78,7.168653e+06
3,000001.SZ,20200707,16.30,16.63,15.03,15.48,15.68,-0.20,-1.2755,3964427.47,6.267920e+06
2,000001.SZ,20200708,15.23,16.00,15.23,15.76,15.48,0.28,1.8088,2631339.16,4.095448e+06
1,000001.SZ,20200709,15.66,15.66,15.31,15.53,15.76,-0.23,-1.4594,2243994.40,3.469517e+06


In [46]:
stock_list.iloc[0]['ts_code']
len(stock_list)

3893

In [47]:
stock_list['ts_code'][0]

'000001.SZ'

In [48]:
# def get_single_daily(single_ts_code):
#     return pro.daily(ts_code=single_ts_code, start_date='20040101', end_date='20200711').sort_values(by=['trade_date'])

# def get_monthly_add(single_ts_code):
#     df_monthly_add = ts.pro_bar(ts_code=single_ts_code, adj='hfq', start_date='20040101', end_date='20200718', freq='M', adjfactor=True, ma=[2, 4])
#     return df_monthly_add

# total_daily = pd.DataFrame()
# n_stock = len(stock_list)
# s_pool = []
# res = []
# i_finish = 0

# with futures.ThreadPoolExecutor(max_workers = 3) as executor:
# #     for single_ts_code in stock_list['ts_code']:
# #         s_pool.append(executor.submit(get_single_daily, single_ts_code))
        
# #     for t in s_pool:
# #         if(t.done()):
# #             res.append(t.result())
# #             i+=1
# #             print("{0}% finished, i = {1} ".format(round(i*100/n_stock), i), end="\r")

# ######################################

# #     s_pool = {executor.submit(get_single_daily, single_ts_code): single_ts_code for single_ts_code in stock_list['ts_code']}
    
# #     for future in futures.as_completed(s_pool):
# #         res.append(future.result())
# #         i_finish+=1
# #         print("{0}% finished, i = {1} ".format(round(i_finish*100/n_stock), i_finish), end="\r")

# ######################################
# #     res = list(executor.map(get_single_daily, stock_list['ts_code']))
    
# #     while(len(res)<n_stock):
# #         i_finish = len(res)
# #         print("{0}% finished, i = {1} ".format(round(i_finish*100/n_stock), i_finish), end="\r")

# ######################################
    
#     s_pool = {executor.submit(get_monthly_add, single_ts_code): single_ts_code for single_ts_code in stock_list['ts_code']}
    
#     for future in futures.as_completed(s_pool):
# #         res.append(future.result())
#         total_daily = total_daily.append(future.result(), ignore_index = True)
#         i_finish+=1
#         print("{0}% finished, i = {1} ".format(round(i_finish*100/n_stock), i_finish), end="\r")

        
# total_daily.shape

In [49]:
def get_monthly_add(single_ts_code):
    df_monthly_add = ts.pro_bar(ts_code=single_ts_code, adj='hfq', start_date='20040101', end_date='20200718', freq='M', adjfactor=True, ma=[2, 4])
    with df_stock:
        global total_daily
        global total_finished
        pd.concat([total_daily, df_monthly_add], ingore_index = True)
        total_finished+=1
        print("{0}% finished, i = {1} ".format(round(total_finished*100/n_stock), total_finished), end="\r")
        
class mTh(threading.Thread):
    def __init__(self, th_name, gl_lock, task_ts_code):
        threading.Thread.__init__(self, name = th_name)
        self.name = th_name
        self.lock = gl_lock
        self.task = task_ts_code
        print("{0} is created".format(self.name))
        
    def run(self):
        global total_daily
        global total_finished
        global n_stock
        global total_stock_ts_code
        
        for single_ts_code in self.task:
            df_single_add = ts.pro_bar(ts_code=single_ts_code, adj='hfq', start_date='20040101', end_date='20200718', adjfactor=True, ma=[2, 4])
            with self.lock:
                total_daily = pd.concat([total_daily, df_single_add], ignore_index = True)
                total_finished+=1
                print("{0} report: {1}% finished, i = {2} ".format(self.name, round(total_finished*100/n_stock), total_finished), end="\r")
#             time.sleep(0.1)
        print("{0} has finished. ".format(self.name))
        
print("ready")

ready


In [50]:
total_daily = pd.DataFrame()
n_stock = len(stock_list)
total_finished = 0
gl_lock = threading.Lock()
th_pool = []

max_n_th = 20
interval = int(math.ceil(len(total_stock_ts_code) / float(max_n_th)))
total_task = [total_stock_ts_code[i:i+interval] for i in range(0, len(total_stock_ts_code), interval)]

for i in range(max_n_th):
    task = total_task[i]
    t = mTh("t{0}".format(i), gl_lock, task)
    th_pool.append(t)
    
for t in th_pool:
    t.start()
    
for t in th_pool:
    t.join()
        
if(total_finished == n_stock):
    print("done")
    
else:
    print("don't match")
    



t0 is created
t1 is created
t2 is created
t3 is created
t4 is created
t5 is created
t6 is created
t7 is created
t8 is created
t9 is created
t10 is created
t11 is created
t12 is created
t13 is created
t14 is created
t15 is created
t16 is created
t17 is created
t18 is created
t19 is created
t19 has finished. nished, i = 2642 
t10 has finished. nished, i = 3046 
t18 has finished. nished, i = 3047 
t17 has finished. nished, i = 3082 
t6 has finished. nished, i = 3153  
t9 has finished. nished, i = 3329  
t11 has finished. nished, i = 3342 
t7 has finished. nished, i = 3424  
t16 has finished. nished, i = 3461 
t8 has finished. nished, i = 3617  
t5 has finished. nished, i = 3621  
t4 has finished. nished, i = 3680  
t15 has finished. nished, i = 3781 
t3 has finished. nished, i = 3799  
t2 has finished. nished, i = 3873  
t0 has finished. inished, i = 3874 
t12 has finished. inished, i = 3887 
t1 has finished. inished, i = 3888 
t14 has finished. inished, i = 3891 
t13 has finished. inishe

In [53]:
total_daily.shape

(8608002, 16)

In [54]:
total_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8608002 entries, 0 to 8608001
Data columns (total 16 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ts_code     object 
 1   trade_date  object 
 2   open        float64
 3   high        float64
 4   low         float64
 5   close       float64
 6   pre_close   float64
 7   change      float64
 8   pct_chg     float64
 9   vol         float64
 10  amount      float64
 11  adj_factor  float64
 12  ma2         float64
 13  ma_v_2      float64
 14  ma4         float64
 15  ma_v_4      float64
dtypes: float64(14), object(2)
memory usage: 1.0+ GB


In [ ]:
# total_daily = pd.DataFrame()
# i = 1
# n_stock = len(stock_list)

# for ts_code_temp in stock_list['ts_code']:
#     single_hist = pro.daily(ts_code=ts_code_temp, start_date='20040101', end_date='20200711').sort_values(by=['trade_date'], inplace=True)
#     total_daily = total_daily.append(single_hist, ignore_index=True)
#     print("{0}% finished, i = {1} ".format(round(i*100/n_stock), i), end="\r")
#     i+=1
#     #time.sleep(0.05)
    
# print("done")

In [27]:
l = list(total_stock_ts_code)
print(len(l))
m = 4
n = int(math.ceil(len(l) / float(m)))
res = [l[i:i+n] for i in range(0, len(l), n)]

z = 0
for i in res:
    print(len(i))
    z+= len(i)

print(z)

3893
974
974
974
971
3893


In [30]:
def test_th(i_input):
    i = i_input
    i_input+=1
    return (i, i_input)

pool = []
res = []
i_finish = 0
# for i in range(4):
#     t = threading.Thread(target=test_th, args=(i, ))
#     pool.append(t)
#     t.start()

# for t in pool:
#     t.join()
    
# print(res)

with futures.ThreadPoolExecutor(max_workers = 2) as executor:
    t_pool = {executor.submit(test_th, i):i for i in range(10)}
    
    print('done')
    for future in futures.as_completed(t_pool):
        res.append(future.result())
        i_finish+=1
        print("i = {0} ".format(i_finish))
        
print(res)

done
i = 1 
i = 2 
i = 3 
i = 4 
i = 5 
i = 6 
i = 7 
i = 8 
i = 9 
i = 10 
[(9, 10), (4, 5), (0, 1), (8, 9), (7, 8), (5, 6), (2, 3), (6, 7), (1, 2), (3, 4)]


In [31]:
def infi_run():
    pid = os.getpid()
    t_counts = 0
    while True:
        print("pid:{0} has been running for {1} s ".format(pid, t_counts) )
        t_counts+=1

class stoppableTh(threading.Thread):
    def __init__(self, daemon=None):
        super().__init__(daemon = daemon)
        self.__is_running = True
        self.daemon = daemon
        
    def terminate(self):
        self.__is_running = False
        
    def run(self):
        pid = os.getpid()
        counts = 0
        while self.__is_running:
            print("pid:{0} has been running for {1} s ".format(pid, counts))
            counts+=1
            time.sleep(1)
            
def call_thread():
    thread = stoppableTh()
    thread.daemon = True
    thread.start()
    time.sleep(5)
    thread.terminate()
